<a href="https://colab.research.google.com/github/dioschuarz/ai_utils/blob/main/HTML_to_PPTX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
"""
Este script é um conversor universal aprimorado que transforma um arquivo HTML em um
slide de apresentação do PowerPoint (.pptx) TOTALmente EDITÁVEL.

Como funciona:
1.  Utiliza um navegador 'headless' (Selenium) para renderizar o HTML e entender
    o layout final dos elementos (posição, tamanho, etc.).
2.  Extrai o conteúdo (texto) e os estilos computados (cor, fonte, borda, etc.) de cada elemento.
3.  Usa a biblioteca python-pptx para recriar cada elemento HTML como um objeto
    nativo e editável do PowerPoint (caixa de texto, forma) na posição correta.

O resultado é um slide que se parece com o HTML, mas é 100% modificável.

Para usar no Google Colab:
1.  Crie um novo notebook.
2.  No painel de arquivos à esquerda, faça o upload do seu arquivo HTML (ex: "status_report.html").
3.  Copie e cole este código em uma célula do notebook.
4.  **IMPORTANTE:** Altere a variável `HTML_FILE_TO_CONVERT` para o nome do seu arquivo.
5.  Execute a célula (pode ser necessário executar mais de uma vez na primeira utilização
    para instalar as dependências).
6.  Baixe o arquivo .pptx gerado.
"""

# Passo 0: Instalação e configuração de dependências
try:
    import selenium
    import pptx
    from bs4 import BeautifulSoup
    import cairosvg
except ImportError:
    print("Instalando bibliotecas necessárias: python-pptx, selenium, beautifulsoup4, cairosvg...")
    import os
    os.system('pip install python-pptx selenium beautifulsoup4 lxml cairosvg > /dev/null')
    print("\nConfigurando o ambiente para o navegador (pode levar um minuto)...")
    os.system('apt-get update > /dev/null')
    os.system('apt install -y chromium-chromedriver > /dev/null')
    os.system('cp /usr/lib/chromium-browser/chromedriver /usr/bin')
    print("\nAmbiente pronto! Por favor, execute a célula novamente para iniciar a conversão.")
    exit()

import os
import re
import io
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN, MSO_VERTICAL_ANCHOR
from pptx.enum.shapes import MSO_SHAPE
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


# --- CONFIGURAções ---
# ⬇️ ALTERE AQUI O NOME DO SEU ARQUIVO HTML ⬇️
HTML_FILE_TO_CONVERT = 'index.html'
# ---------------------------------------------

OUTPUT_PPTX_FILENAME = 'index.pptx'
# Dimensões padrão de um slide 16:9, que correspondem ao viewport
SLIDE_WIDTH_INCHES = 13.333
SLIDE_HEIGHT_INCHES = 7.5
VIEWPORT_WIDTH = 1280 # Alinhado com a proporção 16:9
VIEWPORT_HEIGHT = 720


# --- Funções Auxiliares de Conversão ---

def parse_rgba_color(rgba_string):
    """Converte uma string 'rgba(r, g, b, a)' ou 'rgb(r, g, b)' para uma tupla (r, g, b) e opacidade."""
    try:
        parts = re.findall(r"[\d\.]+", rgba_string)
        if not parts: return (0, 0, 0), 0.0
        r, g, b = int(parts[0]), int(parts[1]), int(parts[2])
        alpha = float(parts[3]) if len(parts) > 3 else 1.0
        return (r, g, b), alpha
    except (IndexError, ValueError):
        return (0, 0, 0), 1.0 # Preto por padrão

def parse_font_family(font_family_string):
    """Extrai o nome da fonte principal de uma string de CSS, removendo aspas."""
    return font_family_string.split(',')[0].strip().replace('"', '').replace("'", "")


def get_elements_data_from_html(file_path):
    """
    Abre o HTML no Selenium, extrai a geometria e os estilos de cada elemento visível.
    Também captura uma imagem do fundo para garantir fidelidade.
    """
    if not os.path.exists(file_path):
        print(f"❌ ERRO: Arquivo '{file_path}' não encontrado. Por favor, faça o upload.")
        return None

    print("🚀 Iniciando o navegador em segundo plano...")
    chrome_options = Options(); chrome_options.add_argument('--headless'); chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage'); chrome_options.add_argument(f'--window-size={VIEWPORT_WIDTH},{VIEWPORT_HEIGHT}')

    driver = webdriver.Chrome(options=chrome_options)
    local_url = f"file://{os.path.abspath(file_path)}"; driver.get(local_url)
    driver.implicitly_wait(3)

    print("📸 Capturando o plano de fundo com fidelidade total...")
    # Adiciona JS para remover a barra de rolagem antes do screenshot
    driver.execute_script("document.documentElement.style.overflow = 'hidden';")

    # Esconde todos os elementos do corpo para tirar uma foto apenas do fundo
    all_elements_js = "return Array.from(document.body.children);"
    all_elements = driver.execute_script(all_elements_js)
    for element in all_elements:
         driver.execute_script("arguments[0].style.visibility = 'hidden';", element)
    background_png = driver.get_screenshot_as_png()
    for element in all_elements:
         driver.execute_script("arguments[0].style.visibility = 'visible';", element)


    print("🔍 Extraindo dados de layout e estilo dos elementos de conteúdo...")
    js_script = """
    const getFontSizeFromClass = (element) => {
        if (element && typeof element.className === 'string') {
            // Regex to find text-[12px] or text-[22.5px] etc.
            const match = element.className.match(/text-\\[(\\d+(\\.\\d+)?)px\\]/);
            if (match && match[1]) {
                return match[1] + 'px';
            }
        }
        return null;
    };

    const container = document.body;
    if (!container) return [];
    const elements = Array.from(container.getElementsByTagName('*'));
    const elementData = [];
    const processedElements = new Set();

    for (const el of elements) {
        if (processedElements.has(el)) continue;

        const rect = el.getBoundingClientRect();
        if (rect.width < 1 || rect.height < 1 || el.offsetParent === null) continue;

        const style = window.getComputedStyle(el);
        const tagName = el.tagName.toLowerCase();

        let data = {
            tag: tagName, x: rect.x, y: rect.y, width: rect.width, height: rect.height,
            background_color: style.backgroundColor, border_radius: style.borderRadius,
            border_top_width: style.borderTopWidth, border_top_color: style.borderTopColor,
            border_right_width: style.borderRightWidth, border_right_color: style.borderRightColor,
            border_bottom_width: style.borderBottomWidth, border_bottom_color: style.borderBottomColor,
            border_left_width: style.borderLeftWidth, border_left_color: style.borderLeftColor,
            padding_top: style.paddingTop,
            padding_right: style.paddingRight,
            padding_bottom: style.paddingBottom,
            padding_left: style.paddingLeft,
            text_align: style.textAlign,
            align_items: style.alignItems,
            justify_content: style.justifyContent,
            display: style.display,
            line_height: style.lineHeight,
            z_index: style.zIndex, svg_content: null, text_runs: [],
            is_bullet_item: tagName === 'li' || style.display === 'list-item'
        };

        if (tagName === 'svg') {
            data.svg_content = el.outerHTML;
            elementData.push(data);
            processedElements.add(el);
        } else {
            let runs = [];
            let isContainerOnly = el.children.length > 0 && Array.from(el.children).every(c => ['block', 'flex', 'grid'].includes(window.getComputedStyle(c).display));

            for (const node of el.childNodes) {
                if (node.nodeType === 3 && node.textContent.trim()) { // TEXT_NODE
                    runs.push({
                        text: node.textContent.trim().replace(/\\s+/g, ' '),
                        color: style.color,
                        font_size: getFontSizeFromClass(el) || style.fontSize,
                        font_weight: style.fontWeight,
                        font_family: style.fontFamily
                    });
                } else if (node.nodeType === 1) { // ELEMENT_NODE
                    const childStyle = window.getComputedStyle(node);
                    if (childStyle.display.includes('inline') && node.textContent.trim()) {
                         runs.push({
                            text: node.textContent.trim().replace(/\\s+/g, ' '),
                            color: childStyle.color,
                            font_size: getFontSizeFromClass(node) || childStyle.fontSize,
                            font_weight: childStyle.fontWeight,
                            font_family: childStyle.fontFamily
                        });
                        node.querySelectorAll('*').forEach(n => processedElements.add(n));
                        processedElements.add(node);
                    }
                }
            }

            // Adiciona o elemento se ele for uma forma visível (tem fundo ou borda) OU se contém texto
            const hasVisibleBg = !data.background_color.includes('rgba(0, 0, 0, 0)');
            const hasVisibleBorder = ['Top', 'Right', 'Bottom', 'Left'].some(side => {
                return parseFloat(style['border' + side + 'Width']) > 0 && !style['border' + side + 'Color'].includes('rgba(0, 0, 0, 0)');
            });

            if ((runs.length > 0 && !isContainerOnly) || hasVisibleBg || hasVisibleBorder) {
                data.text_runs = runs;
                elementData.push(data);
            }
            processedElements.add(el);
        }
    }
    return elementData;
    """
    try:
        elements = driver.execute_script(js_script)
        print(f"✅ Dados de {len(elements)} elementos extraídos com sucesso.")
        return {'elements': elements, 'background_png': background_png}
    except Exception as e:
        print(f"❌ ERRO ao extrair dados dos elementos: {e}")
        return None
    finally:
        driver.quit()

def create_pptx_from_data(extracted_data, output_filename):
    """
    Cria uma apresentação PPTX editável a partir dos dados extraídos dos elementos.
    """
    if not extracted_data or not extracted_data.get('elements'):
        print("Nenhum dado de elemento para processar. Abortando a criação do PPTX.")
        return

    print("🛠️  Construindo o slide editável...")
    prs = Presentation()
    prs.slide_width = Inches(SLIDE_WIDTH_INCHES); prs.slide_height = Inches(SLIDE_HEIGHT_INCHES)
    slide = prs.slides.add_slide(prs.slide_layouts[6]) # Layout em branco

    # Adiciona o fundo capturado
    slide.shapes.add_picture(io.BytesIO(extracted_data['background_png']), 0, 0, width=prs.slide_width, height=prs.slide_height)

    scale_x = prs.slide_width.inches / VIEWPORT_WIDTH
    scale_y = prs.slide_height.inches / VIEWPORT_HEIGHT

    data = extracted_data['elements']
    data.sort(key=lambda el: (0 if el['z_index'] == 'auto' else int(el['z_index']), el['y'], el['x']))

    # --- PASSO 1: Criar todas as formas e imagens (elementos de fundo) ---
    print("➡️  Passo 1 de 2: Desenhando formas, bordas e ícones...")
    for el in data:
        try:
            left=Inches(el['x']*scale_x); top=Inches(el['y']*scale_y)
            width=Inches(el['width']*scale_x); height=Inches(el['height']*scale_y)

            # Processa SVGs como imagens
            if el['svg_content']:
                png_bytes = cairosvg.svg2png(bytestring=el['svg_content'].encode('utf-8'))
                slide.shapes.add_picture(io.BytesIO(png_bytes), left, top, width, height)
                continue

            has_visible_bg = not 'rgba(0, 0, 0, 0)' in el.get('background_color', '')
            border_widths_px = [
                float(el.get(f'border_{side}_width', '0px').replace('px', ''))
                for side in ['top', 'right', 'bottom', 'left']
            ]
            has_visible_border = any(w > 0 for w in border_widths_px)

            # Se for apenas um item de texto sem fundo ou borda, pulamos a criação da forma
            if not has_visible_bg and not has_visible_border:
                 continue

            shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, top, width, height)

            # Preenchimento
            bg_color, bg_alpha = parse_rgba_color(el['background_color'])
            if bg_alpha > 0.05:
                shape.fill.solid(); shape.fill.fore_color.rgb = RGBColor(*bg_color)
            else:
                shape.fill.background()

            # Borda
            if has_visible_border:
                # Usa a cor e a largura da borda mais espessa como referência
                max_width = max(border_widths_px)
                side_index = border_widths_px.index(max_width)
                side_name = ['top', 'right', 'bottom', 'left'][side_index]
                border_color_str = el.get(f'border_{side_name}_color')

                border_color, _ = parse_rgba_color(border_color_str)
                shape.line.color.rgb = RGBColor(*border_color)
                # Conversão 1:1 de px para pt para maior fidelidade visual
                shape.line.width = Pt(max_width)
            else:
                 shape.line.fill.background() # Sem linha/borda

        except Exception as e:
            pass

    # --- PASSO 2: Criar todos os elementos de texto por cima ---
    print("➡️  Passo 2 de 2: Adicionando textos e bullet points...")
    for el in data:
        try:
            if not el['text_runs']: continue

            left=Inches(el['x']*scale_x); top=Inches(el['y']*scale_y)
            width=Inches(el['width']*scale_x); height=Inches(el['height']*scale_y)

            shape = slide.shapes.add_textbox(left, top, width, height)
            tf = shape.text_frame; tf.clear(); tf.word_wrap = True

            # Converte padding do CSS para margens do PPTX para um posicionamento de texto mais fiel
            try:
                # Conversão 1:1 de px para pt para maior fidelidade visual
                tf.margin_top = Pt(float(el.get('padding_top', '0px').replace('px','')))
                tf.margin_right = Pt(float(el.get('padding_right', '0px').replace('px','')))
                tf.margin_bottom = Pt(float(el.get('padding_bottom', '0px').replace('px','')))
                tf.margin_left = Pt(float(el.get('padding_left', '0px').replace('px','')))
            except ValueError:
                 tf.margin_bottom, tf.margin_top, tf.margin_left, tf.margin_right = Pt(0), Pt(0), Pt(0), Pt(0)

            # Alinhamento Vertical
            if (el.get('display') == 'flex' and el.get('align_items') == 'center') or \
               (el.get('text_align') == 'center' and len(el.get('text_runs', [])) > 0):
                tf.vertical_anchor = MSO_VERTICAL_ANCHOR.MIDDLE
            else:
                tf.vertical_anchor = MSO_VERTICAL_ANCHOR.TOP

            p = tf.paragraphs[0]
            p.text = ""

            p.space_before = Pt(0)
            p.space_after = Pt(0)

            # Lógica aprimorada para line-height para maior fidelidade vertical
            line_height_str = el.get('line_height')
            try:
                if line_height_str and line_height_str != 'normal':
                    if 'px' in line_height_str:
                        line_height_px = float(line_height_str.replace('px', ''))
                        first_run_font_size_px = float(el['text_runs'][0]['font_size'].replace('px', ''))
                        if first_run_font_size_px > 0:
                            p.line_spacing = line_height_px / first_run_font_size_px
                        else:
                            p.line_spacing = 1.2 # Fallback
                    else: # É um multiplicador sem unidade (ex: 1.5)
                        p.line_spacing = float(line_height_str)
                else:
                     p.line_spacing = 1.2 # Um default mais realista para 'normal' do que 1.0
            except (ValueError, IndexError):
                p.line_spacing = 1.2 # Fallback

            if el.get('is_bullet_item'):
                p.level = 0

            for i, run_data in enumerate(el['text_runs']):
                run = p.add_run()
                run.text = run_data['text']
                if i < len(el['text_runs']) - 1:
                     run.text += ' '

                font = run.font
                font_color, _ = parse_rgba_color(run_data['color'])
                font.color.rgb = RGBColor(*font_color)
                # Conversão 1:1 de px para pt para maior fidelidade visual
                font.size = Pt(float(run_data['font_size'].replace('px', '')))
                font.bold = int(run_data['font_weight']) >= 600 if run_data['font_weight'].isnumeric() else 'bold' in run_data['font_weight']
                font.name = parse_font_family(run_data.get('font_family', 'Arial'))

            align_map = {'left': PP_ALIGN.LEFT, 'center': PP_ALIGN.CENTER, 'right': PP_ALIGN.RIGHT, 'justify': PP_ALIGN.JUSTIFY}
            p.alignment = align_map.get(el.get('text_align'), PP_ALIGN.LEFT)

        except Exception as e:
            pass

    prs.save(output_filename)
    print(f"\n🎉 Apresentação editável '{output_filename}' criada com sucesso!")

def main():
    """Função principal que orquestra todo o processo."""
    extracted_data = get_elements_data_from_html(HTML_FILE_TO_CONVERT)
    if extracted_data:
        create_pptx_from_data(extracted_data, OUTPUT_PPTX_FILENAME)

if __name__ == '__main__':
    main()



🔪 Dividindo o HTML em slides individuais...
✅ 13 slides foram extraídos para arquivos temporários.

🚢 Processando Slide 1/13...

🚢 Processando Slide 2/13...

🚢 Processando Slide 3/13...

🚢 Processando Slide 4/13...

🚢 Processando Slide 5/13...

🚢 Processando Slide 6/13...

🚢 Processando Slide 7/13...

🚢 Processando Slide 8/13...

🚢 Processando Slide 9/13...

🚢 Processando Slide 10/13...

🚢 Processando Slide 11/13...

🚢 Processando Slide 12/13...

🚢 Processando Slide 13/13...

🛠️  Construindo apresentação final com 13 slides...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apresentação...
➡️  Adicionando slide à apr